# Run the newly generated Smart Village Platform in development


We need to delete the old `smartabyar-smartvillage.yml` configuration file and regenerate it to point to the new route running in development. 

In [ ]:
SITE_HOST_NAME="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get route/vscode-devel -o jsonpath={.spec.host})"
AUTH_CLIENT="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get secret/smartvillage -o jsonpath={.data.CLIENT_ID} | base64 -d)"
AUTH_SECRET="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get secret/smartvillage -o jsonpath={.data.CLIENT_SECRET} | base64 -d)"

rm ~/smartabyar-smartvillage/config/smartabyar-smartvillage.yml

ansible-playbook ~/.ansible/roles/computate.computate_project/install.yml \
  -e SITE_NAME=smartabyar-smartvillage \
  -e SYSTEMD_ENABLED=false \
  -e SITE_PREFIX=/opt/app-root/src \
  -e SOLR_HOST_NAME_COMPUTATE=solr \
  -e SOLR_PORT_COMPUTATE=8983 \
  -e SOLR_SSL_COMPUTATE=false \
  -e SOLR_HOST_NAME=solr \
  -e SOLR_PORT=8983 \
  -e SOLR_SSL=false \
  -e SOLR_URL="http://solr:8983/solr/smartabyar-smartvillage" \
  -e SOLR_URL_COMPUTATE="http://solr:8983/solr/computate" \
  -e ZOOKEEPER_HOST_NAME=zookeeper \
  -e ZOOKEEPER_PORT=2181 \
  -e JDBC_HOST=postgres-smartvillage \
  -e JDBC_PORT=5432 \
  -e POSTGRES_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e ZOOKEEPER_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SOLR_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SITE_BASE_URL="https://${SITE_HOST_NAME}" \
  -e STATIC_BASE_URL="https://${SITE_HOST_NAME}/static" \
  -e AUTH_CLIENT="${AUTH_CLIENT}" \
  -e AUTH_SECRET="${AUTH_SECRET}" \
  -e AUTH_ROLE_ADMIN="Public" \
  -e ENABLE_KAFKA=false \
  -e ENABLE_IMPORT_DATA=false
echo DONE

Open a new terminal and run the command below to run your newly generated Smart Village Platform in development. 

```bash
~/smart-device-api-generation-hackathon/run.sh
```

### Open Service, Route, and NetworkPolicy for live cloud development



Create the OpenShift Service definition below to open up the port 12080 of the notebook to network traffic in the namespace. 

In [ ]:
echo "
kind: Service
apiVersion: v1
metadata:
  name: vscode-devel
  namespace: $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace)
spec:
  ports:
    - protocol: TCP
      port: 12080
      targetPort: 12080
  type: ClusterIP
  selector:
    notebook-name: vscode
" | oc apply -f -


Create the OpenShift NetworkPolicy definition below allow access from the route to the port 12080 of the notebook pod. 

In [ ]:
echo "
kind: NetworkPolicy
apiVersion: networking.k8s.io/v1
metadata:
  name: vscode-devel
spec:
  podSelector:
    matchLabels:
      notebook-name: vscode
  ingress:
    - ports:
        - protocol: TCP
          port: 12080
  policyTypes:
    - Ingress
status: {}
" | oc apply -f -


Create the OpenShift Route definition below to expose the running Smart Village application in development to the Internet. 

In [ ]:
echo "
kind: Route
apiVersion: route.openshift.io/v1
metadata:
  name: vscode-devel
  labels:
    notebook-name: vscode
spec:
  to:
    kind: Service
    name: vscode-devel
    weight: 100
  port:
    targetPort: 12080
  tls:
    termination: edge
  wildcardPolicy: None
" | oc apply -f -

echo "https://$(oc get route/vscode-devel -o jsonpath={.spec.host})/PhotovoltaicMeasurement"

## Visit your own instance of the Smart Village Platform

Click on the link above to access the dashboard for PhotovoltaicMeasurement records in your own instance of the Smart Village Platform. 